Importing libraries

In [ ]:
import json
from datetime import datetime

Defining Classes

In [ ]:
class Sensor:
    """Represents a sensor associated with an EEG amplifier."""

    def __init__(self, tag):
        """Initializes the sensor with a position tag."""
        self.tag = tag

    def __str__(self):
        """Returns a string representation of the sensor."""
        return self.tag

class Amplifier:
    """
    Represents an EEG amplifier.

    Attributes:
        serial_number (str): Unique identifier for the amplifier.
        model (str): Model string of the amplifier.
        manufacturer (str): Manufacturer of the amplifier.
        maintenance_date (str): Date for the next planned maintenance.
        sampling_rate (int): Sampling rate in Hz (256, 512, or 1024).
        gain (int): Current gain level (1 to 100).
        is_powered_on (bool): Operational status of the amplifier.
        sensors (list): List of associated sensors.
    """

    def __init__(self, serial_number, model, manufacturer, maintenance_date, sampling_rate, gain):
        """Initializes the amplifier with its properties."""
        self.serial_number = serial_number
        self.model = model
        self.manufacturer = manufacturer
        self.maintenance_date = maintenance_date
        self.sampling_rate = sampling_rate  # in Hz
        self.gain = gain  # within [1, 100]
        self.is_powered_on = False
        self.sensors = []

    def add_sensor(self, sensor):
        """Adds a sensor to the amplifier."""
        self.sensors.append(sensor)

    def remove_sensor(self, sensor):
        """Removes a sensor from the amplifier."""
        self.sensors.remove(sensor)

    def update_maintenance_date(self, new_date):
        """Updates the maintenance date of the amplifier.

        Args:
            new_date (str): The new maintenance date in 'YYYY-MM-DD' format.

        Raises:
            ValueError: If the date format is invalid.
        """
        try:
            datetime.strptime(new_date, '%Y-%m-%d')
            self.maintenance_date = new_date
        except ValueError:
            raise ValueError("Invalid date format. Please use 'YYYY-MM-DD'.")

    def power_on(self):
        """Powers on the amplifier."""
        self.is_powered_on = True

    def power_off(self):
        """Powers off the amplifier."""
        self.is_powered_on = False

    def set_sampling_rate(self, rate):
        """Sets the sampling rate for the amplifier.

        Args:
            rate (int): The sampling rate to set.

        Raises:
            ValueError: If the rate is not 256, 512, or 1024.
        """
        if rate in [256, 512, 1024]:
            self.sampling_rate = rate
        else:
            raise ValueError("Invalid sampling rate. Must be 256, 512, or 1024 Hz.")

    def set_gain(self, gain):
        """Sets the gain for the amplifier.

        Args:
            gain (int): The gain to set.

        Raises:
            ValueError: If the gain is not between 1 and 100.
        """
        if 1 <= gain <= 100:
            self.gain = gain
        else:
            raise ValueError("Gain must be between 1 and 100.")

    def __str__(self):
        """Returns a string representation of the amplifier."""
        return (f"Serial: {self.serial_number}, Model: {self.model}, Manufacturer: {self.manufacturer}, "
                f"Next Maintenance: {self.maintenance_date}, Sampling Rate: {self.sampling_rate}, "
                f"Gain: {self.gain}, Powered On: {self.is_powered_on}, Sensors: {[str(s) for s in self.sensors]}")

EEG Amplifier and Control System Class

In [ ]:
class EEGAmplifierControlSystem:
    """
    Manages a collection of EEG amplifiers.

    Attributes:
        amplifiers (list): List of registered amplifiers.
    """

    def __init__(self):
        """Initializes the EEG Amplifier Control System."""
        self.amplifiers = []

    def add_amplifier(self, amplifier):
        """Adds an amplifier to the system."""
        self.amplifiers.append(amplifier)

    def remove_amplifier(self, serial_number):
        """Removes an amplifier from the system by its serial number."""
        self.amplifiers = [amp for amp in self.amplifiers if amp.serial_number != serial_number]

    def list_amplifiers(self):
        """Lists all registered amplifiers."""
        for amp in self.amplifiers:
            print(amp)

    def search_amplifier(self, query):
        """Searches for an amplifier by serial number, model, or manufacturer.

        Args:
            query (str): The serial number, model, or manufacturer to search for.
        """
        found = False
        for amp in self.amplifiers:
            if (amp.serial_number == query or
                    amp.model == query or
                    amp.manufacturer == query):
                print(amp)
                found = True
        if not found:
            print("No amplifier found with the given query.")

    def toggle_power(self, serial_number):
        """Toggles the power status of an amplifier by its serial number.

        Args:
            serial_number (str): The serial number of the amplifier to toggle.
        """
        for amp in self.amplifiers:
            if amp.serial_number == serial_number:
                if amp.is_powered_on:
                    amp.power_off()
                else:
                    amp.power_on()
                print(f"Amplifier {serial_number} powered {'on' if amp.is_powered_on else 'off'}.")
                return
        print("Amplifier not found.")

    def save_state(self, filename):
        """Saves the current state of amplifiers to a file in JSON format.

        Args:
            filename (str): The name of the file to save the state.
        """
        with open(filename, 'w') as file:
            json.dump([amp.__dict__ for amp in self.amplifiers], file)
        print("State saved to", filename)

    def load_state(self, filename):
        """Loads the state of amplifiers from a file in JSON format.

        Args:
            filename (str): The name of the file to load the state from.
        """
        try:
            with open(filename, 'r') as file:
                amplifiers_data = json.load(file)
                self.amplifiers = [Amplifier(**data) for data in amplifiers_data]
            print("State loaded from", filename)
        except FileNotFoundError:
            print("File not found. Starting with an empty amplifier list.")
        except json.JSONDecodeError:
            print("Error reading the JSON file. Please check the file format.")

User Interface Function

In [ ]:
def display_menu():
    """Displays the menu options for the EEG Amplifier Control System."""
    print("\nEEG Amplifier Control System")
    print("1. Add Amplifier")
    print("2. Remove Amplifier")
    print("3. List Amplifiers")
    print("4. Search Amplifier")
    print("5. Set Gain")
    print("6. Set Sampling Rate")
    print("7. Toggle Power")
    print("8. Update Maintenance Date")
    print("9. Add Sensor")
    print("10. Remove Sensor")
    print("11. Save State")
    print("12. Load State")
    print("13. Exit")

Main Function

In [ ]:
def main():
    """Main function to run the EEG Amplifier Control System."""
    system = EEGAmplifierControlSystem()
    system.load_state('amplifiers.json')  # Load the state from a file on startup

    while True:
        display_menu()
        choice = input("Enter your choice: ")

        try:
            if choice == '1':
                serial_number = input("Enter serial number: ")
                model = input("Enter model: ")
                manufacturer = input("Enter manufacturer: ")
                maintenance_date = input("Enter next maintenance date (YYYY-MM-DD): ")
                sampling_rate = int(input("Enter sampling rate (256, 512, 1024): "))
                gain = int(input("Enter gain (1-100): "))
                amplifier = Amplifier(serial_number, model, manufacturer, maintenance_date, sampling_rate, gain)
                system.add_amplifier(amplifier)
                print("Amplifier added.")

            elif choice == '2':
                serial_number = input("Enter serial number to remove: ")
                system.remove_amplifier(serial_number)
                print("Amplifier removed.")

            elif choice == '3':
                system.list_amplifiers()

            elif choice == '4':
                query = input("Enter serial number, model, or manufacturer to search: ")
                system.search_amplifier(query)

            elif choice == '5':
                serial_number = input("Enter serial number to set gain: ")
                gain = int(input("Enter gain (1-100): "))
                for amp in system.amplifiers:
                    if amp.serial_number == serial_number:
                        amp.set_gain(gain)
                        print("Gain updated.")
                        break
                else:
                    print("Amplifier not found.")

            elif choice == '6':
                serial_number = input("Enter serial number to set sampling rate: ")
                rate = int(input("Enter sampling rate (256, 512, 1024): "))
                for amp in system.amplifiers:
                    if amp.serial_number == serial_number:
                        amp.set_sampling_rate(rate)
                        print("Sampling rate updated.")
                        break
                else:
                    print("Amplifier not found.")

            elif choice == '7':
                serial_number = input("Enter serial number to toggle power: ")
                system.toggle_power(serial_number)

            elif choice == '8':
                serial_number = input("Enter serial number to update maintenance date: ")
                new_date = input("Enter new maintenance date (YYYY-MM-DD): ")
                for amp in system.amplifiers:
                    if amp.serial_number == serial_number:
                        amp.update_maintenance_date(new_date)
                        print("Maintenance date updated.")
                        break
                else:
                    print("Amplifier not found.")

            elif choice == '9':
                serial_number = input("Enter serial number to add sensor: ")
                sensor_tag = input("Enter sensor tag: ")
                sensor = Sensor(sensor_tag)
                for amp in system.amplifiers:
                    if amp.serial_number == serial_number:
                        amp.add_sensor(sensor)
                        print("Sensor added.")
                        break
                else:
                    print("Amplifier not found.")

            elif choice == '10':
                serial_number = input("Enter serial number to remove sensor: ")
                sensor_tag = input("Enter sensor tag to remove: ")
                for amp in system.amplifiers:
                    if amp.serial_number == serial_number:
                        for sensor in amp.sensors:
                            if sensor.tag == sensor_tag:
                                amp.remove_sensor(sensor)
                                print("Sensor removed.")
                                break
                        else:
                            print("Sensor not found.")
                        break
                else:
                    print("Amplifier not found.")

            elif choice == '11':
                filename = input("Enter filename to save state: ")
                system.save_state(filename)

            elif choice == '12':
                filename = input("Enter filename to load state: ")
                system.load_state(filename)

            elif choice == '13':
                print("Exiting the system.")
                break

            else:
                print("Invalid choice. Please try again.")

        except Exception as e:
            print(f"An error occurred: {e}")

Running the system

In [ ]:
if __name__ == "__main__":
    main()

File not found. Starting with an empty amplifier list.

EEG Amplifier Control System
1. Add Amplifier
2. Remove Amplifier
3. List Amplifiers
4. Search Amplifier
5. Set Gain
6. Set Sampling Rate
7. Toggle Power
8. Update Maintenance Date
9. Add Sensor
10. Remove Sensor
11. Save State
12. Load State
13. Exit
Enter your choice: 1
Enter serial number: ABC
Enter model: XYZ
Enter manufacturer: BIC
Enter next maintenance date (YYYY-MM-DD): 2024-01-23
Enter sampling rate (256, 512, 1024): 256
Enter gain (1-100): 65
Amplifier added.

EEG Amplifier Control System
1. Add Amplifier
2. Remove Amplifier
3. List Amplifiers
4. Search Amplifier
5. Set Gain
6. Set Sampling Rate
7. Toggle Power
8. Update Maintenance Date
9. Add Sensor
10. Remove Sensor
11. Save State
12. Load State
13. Exit
Enter your choice: 1
Enter serial number: 2
Enter model: ABC
Enter manufacturer: XYZ
Enter next maintenance date (YYYY-MM-DD): 2024-01-23
Enter sampling rate (256, 512, 1024): 256
Enter gain (1-100): 65
Amplifier adde